### Numerical solution to Viscous Burger equation using Crank-Nicolson methods
Viscous Burger's equation
$$ u_t + u u_x - \nu u_{xx} = 0, \nu = \frac{\mu}{50 \pi}, \text{ where } \mu \in \{1,1.25,...,7.5\} \text{ in this notebook.}$$
$ -1 < x < 1, 0 < t < 1$

Boundary conditions: $ u(-1,t) = u (1,t) = 0$

Initial conditions: $ u(x,0) = -sin(\pi x)$

#### References
[1] Pandey, K. & Verma, Lajja. (2011). A Note on Crank-Nicolson Scheme for Burgers' Equation. Applied Mathematics. 02. 10.4236/am.2011.27118. 

[2] Wani, Sachin & Thakar, Sarita. (2013). Crank-Nicolson Type Method for Burgers Equation. International Journal of Applied Physics and Mathematics. 3. 324-328. 10.7763/IJAPM.2013.V3.230. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
def A(u_in,k,h,v):
    return(((k*u_in)/(4*h)) - ((k*v)/(2*(h**2))))

def B(u_i1n, u_I1n,k,h,v):
    return((k*(u_i1n - u_I1n)/(4*h)) + ((k*v)/(h**2)) + 1)

def C(u_in,k,h,v):
    return(-((k*u_in)/(4*h)) - ((k*v)/(2*(h**2))))

def D(u_i1n, u_in, u_I1n,k,h,v):
    return(((k*v*u_i1n)/(2*(h**2))) + u_in*(1-((k*v)/(h**2))) + ((k*v*u_I1n)/(2*(h**2))))

In [3]:
def Crank_Nicolson(X,N,k,h,v):
    listB = []
    for i in range(N-2):
        listB.append(B(X[i+2],X[i],k,h,v))
    BB = np.diag(listB)
    #print(BB)
    
    listA = []
    for i in range(N-3):
        listA.append(A(X[i+1],k,h,v))
    AA = np.diag(listA,1)
    #print(AA)
    
    listC = []
    for i in range(N-3):
        listC.append(C(X[i+1],k,h,v))
    CC = np.diag(listC,-1)
    #print(CC)
    
    M = AA + BB + CC
    #print(M)
    #print(M.shape)
    
    listD = []
    listD.append(D(X[2],X[1],X[0],k,h,v))
    for i in range(1,N-3):
        listD.append(D(X[i+2],X[i+1],X[i],k,h,v))
    listD.append(D(X[N-4],X[N-3],X[N-2],k,h,v))
    DD = np.array(listD).reshape(-1,1)
    #print(DD.shape)
    
    uu = np.linalg.solve(M, DD)
    
    # plt.plot(t[1:N-1],uu)
    #print(np.max(uu))
    return(uu)

In [4]:
def viscous_burger(mu):
    # initialize conditions
    N = 1024
    h = 2/N # Dx
    k = 1/N # Dt
    x = np.arange(-1,1+h,h)
    t = np.arange(0,1+k,k)
    
    boundaryCond = [0,0]
    initialCond = - np.sin(np.pi*x)
    
    # set up the solution mesh
    n = len(x) # dimension of parametrized spatial space
    m = len(t) # dimension of parametrized time space 
    mesh = np.zeros((n,m))
    mesh[0,:] = boundaryCond[0]
    mesh[-1,:] = boundaryCond[1]
    mesh[:,0] = initialCond
    
    # set up the equation
    v = mu/(50*np.pi)
    
    for i in tqdm(range(mesh.shape[1])):
        uu = Crank_Nicolson(mesh[:,i],N,k,h,v)
        uu = uu.flatten()
    
        if np.max(uu) > 100:
            print(np.max(uu))
            break
        if i <= N-2:
            mesh[1:N-1,i+1] = uu
            
    return(mesh)

In [5]:
N = 1024
h = 2/N # Dx
k = 1/N # Dt
x = np.arange(-1,1+h,h)
t = np.arange(0,1+k,k)

In [6]:
Mu = np.arange(1,7.75,0.25)
snapshot = viscous_burger(mu=1)
for mu in Mu[1:]:
    print(mu)
    sol_mesh = viscous_burger(mu)
    snapshot = np.append(snapshot, sol_mesh, axis=1)

100%|██████████████████████████████████████████████████████████████████████████████| 1025/1025 [01:31<00:00, 11.24it/s]


1.25


100%|██████████████████████████████████████████████████████████████████████████████| 1025/1025 [03:15<00:00,  5.24it/s]


1.5


100%|██████████████████████████████████████████████████████████████████████████████| 1025/1025 [03:40<00:00,  4.64it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████| 1025/1025 [01:42<00:00, 10.02it/s]


7.25


100%|██████████████████████████████████████████████████████████████████████████████| 1025/1025 [01:47<00:00,  9.51it/s]


7.5


100%|██████████████████████████████████████████████████████████████████████████████| 1025/1025 [01:52<00:00,  9.08it/s]


In [7]:
df = pd.DataFrame(snapshot)
print(df.shape)
df.to_csv('snapshot.csv',index=False)

(1025, 27675)


In [8]:
df.to_csv('full-order_sol.csv',index=False)